# setup

In [1]:
import torch

from PIL import Image
from torchvision import transforms

from datasets import load_dataset
from utils import transform, collate_fn, Imagenet_h

from tqdm import tqdm

In [2]:
BATCH_SIZE = 256
EPOCHS = 1

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# data

In [4]:
trn = load_dataset("evanarlian/imagenet_1k_resized_256", split = 'train')
val = load_dataset("evanarlian/imagenet_1k_resized_256", split = 'val')
trn, val

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

(Dataset({
     features: ['image', 'label'],
     num_rows: 1281167
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 50000
 }))

In [5]:
trn = trn.with_transform(transform)
val = val.with_transform(transform)

trn.num_rows / BATCH_SIZE, val.num_rows / BATCH_SIZE

(5004.55859375, 195.3125)

In [6]:
H_trn = torch.load('output/rn34_H_train.pt')
H_val = torch.load('output/rn34_H_val.pt')

# H_trn = torch.load('output/rn18rn34_H_train.pt')
# H_val = torch.load('output/rn18rn34_H_val.pt')

In [7]:
trn = Imagenet_h(trn, H_trn)
val = Imagenet_h(val, H_val)

In [8]:
trn_loader = torch.utils.data.DataLoader(trn, batch_size = BATCH_SIZE, shuffle = True)
val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE)

trn.__len__() / BATCH_SIZE, val.__len__() / BATCH_SIZE

(5004.55859375, 195.3125)

# model

In [9]:
linear_probe = torch.nn.Sequential(
    torch.nn.Linear(in_features = 512, out_features = 1000, bias = True),
    torch.nn.Softmax(),
)
linear_probe = linear_probe.to(device)

In [10]:
# for name, param in linear_probe.named_parameters():
#     if name not in ['2.weight', '2.bias']:
#         param.requires_grad = False

# parameters = list(filter(lambda p: p.requires_grad, linear_probe.parameters()))
# assert len(parameters) == 2  # 2.weight, 2.bias

# train

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(linear_probe.parameters())

In [12]:
linear_probe.train()
for epoch in (range(EPOCHS)):
    for x, y in tqdm(trn_loader):
        x = x.to(device)
        y = y.to(device)

        output = linear_probe.forward(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  0%|          | 0/5005 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  1%|          | 29/5005 [01:11<3:25:20,  2.48s/it]


KeyboardInterrupt: 

# eval

In [ ]:
total = 0
correct = 0

linear_probe.eval()
with torch.no_grad():
    for x, y in tqdm(val_loader):
        x = x.to(device)
        y = y.to(device)

        output = linear_probe.forward(x)
        _, predicted = torch.max(output.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
1 - correct / total